In [1]:
# Imports
import os
import requests
import pandas as pd
import datetime
from twilio.rest import Client

In [2]:
# GLOBAL API Setup
ALPHA_API_KEY = os.environ.get('ALPHA_API_KEY')
NEWS_API_KEY = os.environ.get('NEWS_API_KEY')
TWILIO_ACCOUNT_SID = os.environ.get('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.environ.get('TWILIO_AUTH_TOKEN')
FROM_WPP = os.environ.get('FROM_WPP')
TO_WPP = os.environ.get('TO_WPP')
ALPHA_URL = 'https://www.alphavantage.co/query?'
NEWS_URL = 'https://newsapi.org/v2/everything'


In [18]:

# Stock variation calculation
def get_stock_variation(stock_symbol) -> bool:
    """Fetch stock variation percentage and check if it exceeds 5%."""
    alpha_params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': stock_symbol.upper(),
        'apikey': ALPHA_API_KEY
    }
    
    response = requests.get(ALPHA_URL, params=alpha_params)
    response.raise_for_status()
    raw_data = response.json()
    data = raw_data["Time Series (Daily)"]
    
    # Get sorted available dates from API
    dates = sorted(data.keys(), reverse=True)
    
    try:
        yesterday_date = dates[0]
        two_days_ago_date = dates[1]
    except IndexError:
        print("Not enough data to calculate stock variation")
        return False
    
    # Fetch close prices
    yesterday_close = float(data[yesterday_date]["4. close"])
    two_days_close = float(data[two_days_ago_date]["4. close"])
    
    percentage_difference = (yesterday_close - two_days_close) / yesterday_close * 100
    print(f'Stock price variation:{percentage_difference}')
    
    if abs(percentage_difference) >= 5:
        print("The variation is more than 5%")
        return True
    else:
        print("The variation is within 5%")
        return False

In [4]:
# Fetch news articles
def get_news(query) -> list:
    """Retrieve top 3 news articles based on the query."""
    
    yesterday_date = datetime.datetime.now() - datetime.timedelta(days=1)
    news_params = {
        'apiKey': os.environ.get('NEWS_API_KEY'),
        'q': query,
        'from': yesterday_date,
        'language': 'en',
        'sortBy':'relevancy'
    }
    
    response = requests.get(NEWS_URL, news_params)
    response.raise_for_status()
    data = response.json() # o: python dict
    # print(data)
    # print(type(data))

    return data['articles'][0:3]

In [5]:
# Send wpp message
def set_wpp_message(message):
    """Send a WhatsApp message using Twilio."""
    client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
    msn = client.messages.create( 
        body=message,
        from_=FROM_WPP,
        to=TO_WPP,
        )
    return msn.status

In [10]:
# TEST
stock_price_variation = get_stock_variation("AAPL")
print(stock_price_variation)
news = get_news("apple")
print(news)
message= "Holi, testing"
msn = set_wpp_message(message)

-0.395008528593229
The variation is within 5%
False
[{'source': {'id': None, 'name': '9to5Mac'}, 'author': 'Michael Burkhardt', 'title': 'Gurman: Apple Stores performing ‘marketing updates’ next week', 'description': 'According to a tweet from Bloomberg’s Mark Gurman, Apple will be performing some “merchandise/floor marketing updates” at its retail stores next week. While specific details aren’t confirmed, Apple is likely launching the new Black Unity Apple Watch band for …', 'url': 'https://9to5mac.com/2025/01/25/apple-store-marketing-updates-gurman/', 'urlToImage': 'https://i0.wp.com/9to5mac.com/wp-content/uploads/sites/6/2024/09/apple-fifth-ave-apple-intelligence-iphone-16.jpg?resize=1200%2C628&quality=82&strip=all&ssl=1', 'publishedAt': '2025-01-25T22:47:10Z', 'content': 'According to a tweet from Bloomberg’s Mark Gurman, Apple will be performing some “merchandise/floor marketing updates” at its retail stores next week. While specific details aren’t confirmed, Apple i… [+1123 chars

In [19]:
def main():
    stock_price_variation = get_stock_variation("AAPL")
    if stock_price_variation:
        news = get_news("apple")
        for article in news:
            message = f"Headline: {article['title']}\nBrief: {article['description']}\nLink: {article['url']}"
            set_wpp_message(message)
    else:
        print("No stock variation detected")

In [20]:
main()

Stock price variation:-0.395008528593229
The variation is within 5%
No stock variation detected
